# Entropy and Decision Trees

In this exercise we shall look at the concepts of 'entropy' and 'information gain' and how they can be employed in coding a 'decision tree', which can be used to classify samples.

If you want to understand the extremely significant but enigmatic concept of 'entropy' I suggest the following:
[The Science of Information: From Language to Black Holes, by Benjamin Schumacher](https://www.amazon.co.uk/Science-Information-Language-Black-Holes/dp/B07K8ZBG14)

## The data

We shall apply the decision tree technique to the well known 
<a href="https://en.wikipedia.org/wiki/Iris_flower_data_set">Iris flower dataset</a> that contains information about properties and classification of iris flowers. Briefly, there are three types of Iris flower. The dataset contains measurements for many individual flowers, along with a classification stating which of the three types of Iris each flower is. Our aim is to learn how the flower type can be predicted from the measurements given.  

The Python library scikit-learn includes several datasets that you can practice with. You can read about the Iris dataset and others [here](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets). The data provided is very clean so there is no extra work to do before we can start our analysis. That's really useful when learning a new technique but remember that when you come to use real data there is likely to be a significant amount of initial effort required to deal with missing or incomplete data, poor formatting and wrong datatypes etc.

Let's begin by loading the data. 

In [ ]:
from sklearn import datasets   # load the sklearn datasets
iris = datasets.load_iris()    # load the "iris" dataset.

The dataset that has been assigned to variable "iris" is a somewhat complex Python object. As well as arrays of numerical data that give measurements of a set of samples, it includes other information regarding the naming of the data, and the desired classification of the data into "target" classes.

The most important attributes are described in the following table.

|attribute name|description|
| :---: | :--- |
|data|the actual data|
|target|the correct classification|
|target_names|the names of each classification|
|feature_names|names of the columns|
|DESCR|a summary of the dataset object|

The data and target are provided as numpy arrays.

To see the full data structure, run the following cell.

In [ ]:
iris

The data is a numpy array by default, or a pandas dataframe if `as_frame=False` is passsed into the `load_iris()` function.
Let us have a look at some of the data in the iris dataset.
We can get the first 5 items of the data array by doing:

In [ ]:
iris.data[0:5]

These numbers correspond to the values of certain features. We can look at the feature names as follows:

In [ ]:
iris.feature_names

The target classification is a single value (0, 1, or 2) for each item. 
The target_names array gives the meaning of these values.

In [ ]:
iris.target

In [ ]:
iris.target_names

So, for example, the first entry has target 0 so these measurements were from a flower of type 'setsoa'.
The following function prints out all the relevant information for a single item of data in a nice readable format.

In [ ]:
def print_row(iris, i):
    print("Data row {}".format(i))
    values = iris.data[i]
    for feature_index, feature_name in enumerate(iris.feature_names):
        print('{}: {}'.format(feature_name, values[feature_index]))
    print('Target: {} (code {})'.format(iris.target_names[iris.target[i]], iris.target[i]))

print_row(iris, 103)

## Decision trees

A **decision tree** is a classifier, which  decides the classification of entities based on a sequence of tests. In this example, we want to create a decision tree that will classify iris flowers into one of three types based on tests relating to the four measurements provided in the dataset.

Scikit-learn offers a huge range of machine learning functionality, including a decision tree class and the ability to learn decision trees. In fact, we only need to write a few of lines of code! 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# learn a decision tree 
clf = DecisionTreeClassifier().fit(iris.data, iris.target)

To see what this has created we can use the built-in functionality to print out the decision tree.
Spend some time studying the decision tree diagram that is printed out. 

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# display the decision tree
plt.figure(figsize=(16,12))
plot_tree(clf)
plt.show()

In the diagram, each rectangle is a 'node' in the tree.
Some nodes have arrows to two other nodes, these are called its 'children'. 
A node which has no outgoing arrows is called a 'leaf'.
A non-leaf node in a decision tree represents a test that can be applied to a sample.

The first line written at each non-leaf node describes the test associated with that node. For example, `X[2] <= 2.45` means the first node applies the test of whether the petal length measurement (feature 2 in our dataset) is less than or equal to 2.45cm. 

Consider the example piece of data 
```
Data row 103
sepal length (cm): 6.3
sepal width (cm): 2.9
petal length (cm): 5.6
petal width (cm): 1.8
Target: virginica (code 2)
```
We can use the tests in the tree to classify it. 
The petal length is not less than 2.45cm. 
Because the test from the first node is false in this case we should move to the right-hand child.
In the case that the test was true we would have moved to the left-hand child. 
The test in the next node is `X[3] <= 1.75`. The petal width is not less than 1.75 for this sample so again we move to the right-hand child. 
Apply the test in this node, you should see that we move again to the right-hand child, which is a leaf node. 
Now there are no more tests to apply. 

The last line at each node shows the number of samples from the training data of each classification for which applying the tests as described would lead to this node.
At the leaf nodes all the samples have the same classification.
If a sample reaches this node we know that it belongs to this classification. 

The method `predict()` will apply the tests to a list of samples and return a list of classifications.

In [ ]:
# get the classification for sample 103
clf.predict([iris.data[103]])

Of course, this sample was used in defining the decision tree and we already know its classification, so being able to 'predict' the classification isn't very useful! 
In general, as with any form of machine learning, we would aim to learn a model which can be applied to data that it hasn't been trained on. 

The rest of this lesson will show how to implement a simple decision tree learning algorithm yourself. 
It's often much easier and better to use well maintained packages like scikit-learn for machine learning rather than try to write your own from scratch. 
However, implementing your own algorithm, even though it might be imperfect, is one way to gain a deeper insight into the theoretical basis for a technique.

## The Entropy Equation

Before we can build a decision tree we need to understand the concept of entropy.
Entropy is the average amount of information given by an event, when considering all possible outcomes.
For example, a single flip of an unbiased coin gives one bit of information.
The outcome can be represented by a single bit (a 0 or 1) and we have no reason to expect one outcome over the other so this one bit is giving us the maximum possible new information that it can. 
At the opposite extreme, the outcome of flipping a coin which always lands the same way can also be represented in a single bit of data, but because we already know what the outcome will be before looking that bit carries no new information. 

Another way to think about entropy is the amount of diversity in a dataset. 
If every value is the same then each individual data point gives no new information.
So a collection of data points has lower entropy if most items belong to the same class (or to a small number of classes) and higer entropy if the data points show a great deal of variety and are more evenly split between the possible classes. 

More specifically, suppose we have a set of $N$ events (e.g. coin flips) that can be divided into $k$ classes (e.g. heads or tails) and the number of entities in class $i$ is $n_i$ (so $\sum_{i=1}^k n_i = N$). Then the proportion $P_i$ of entities that are in class $i$
is $\frac{n_i}{N}$. 

The _entropy_ $S$ of this classification is given by the equation
$$
S= - \sum_{i=1}^k P_i \log_2 (P_i)
$$

We can also write this as:
$$
S= \sum_{i=1}^k P_i \log_2 \left(\frac{1}{P_i}\right)
$$

Here the $\log_2 \left(\frac{1}{P_i}\right)$ term gives the average number of _bits_ that would be needed to specify that an entitiy lies within a $P_i$ sized portion of the set.

The following cell defines a function to calculate entropy from a list specifying how often multiple outcomes occur.
We then use this function to calculate and plot the entropy for events in which there are two outcomes and the first happens $x$ times out of 100 for $x$ ranging in value from 0 to 100. 
As expected, an event where each outcome happens 50 times out of 100 has entropy 1. 

In [ ]:
import math

def calculate_entropy_from_outcome_distribution(number_vector):
    """
        Calculate entropy given a list of the number of outcomes in each category.
        
        Parameters:
            number_vector list(ints): Numbers of outcomes for different categories.
            
        Returns:
            A float representing the entropy of an event with distribution as number_vector
            
        len(number_vector) is the number of different possible outcomes.
        number_vector[i] is the number of events with outcome i.
        sum(number_vector) is the total number of events.
        Therefore number_vector[i]/sum(number_vector) is the probability of outcome i.
    """
    total = sum(number_vector) # N, total number of events 
    entropy = 0 # initialise
    
    for number in number_vector: # n_i, number of outcomes of type i
        if number == 0:
            continue # outcome did not occur, has no effect on entropy
        proportion = number/total # P_i
        entropy -= proportion * math.log(proportion, 2) # P_i * log_2(P_i)
    
    return entropy

In [ ]:
import matplotlib.pyplot as plt  

# list of tuples of the form (number of events with outcome A, number of events with outcome B)
# with 100 total outcomes recorded
xvals = [(x, 100-x) for x in range(0,101)] 

# Calculate entropy for each outcome possibility
yvals = [calculate_entropy_from_outcome_distribution(outcome_tuple) for outcome_tuple in xvals]  

# plot freq of outcome A vs entropy
fig, ax = plt.subplots()
ax.plot(xvals, yvals) 

ax.grid()
ax.set_title('Entropy of events with exactly two outcomes')
ax.set_xticks(range(0,101, 10))
ax.set_xlabel('% of events with outcome A')
ax.set_ylabel('Entropy')
plt.show()

## Defining a  Decision Tree
A simple binary decision tree can be defined by the following recursive definition:

*    `DT => [Test, DT, DT]`
*    `DT => C1 | ... | Cn`

The first case corresponds to the application of a test to an entity. According to whether the test is positive or negative we get one of two decision trees, which can then be again applied to the entity. 
The second clause represents the end of the decision process, where we assign a class to the sample.

The challenge when creating a decision tree is to find 'good' tests. 
We aim to use as few tests as possible to classify an entity.
To decide which tests to use we take some training data and consider all the tests which could split the samples into two parts.
Then we find the entropy of each part with respect to the classifications we want to recognise.
If the entropy is low in a collection of samples then this means most of the samples belong to the same class.
If both parts have low entropy that suggests the test was good at distinguishing between the different possible classes. 
We repeat this process until our training data has been split into collections which each contain entities belonging to a single class.

We can now define functions to build a decision tree in this way and use it to classify a sample. 

### Sample object
Sample objects will contain a set of properties and their classification.
A nice structure that Python provides is a named tuple.
This is just a tuple in which each item can be referenced by a name as well as its index, which can make code a bit more readable.
It's a lightweight alternative to a very simple class.

In [ ]:
from collections import namedtuple 

Sample = namedtuple('Sample', ['properties', 'class_index'])

Now create a sample object for each record in the iris data.

In [ ]:
iris_samples = [Sample(iris.data[i], iris.target[i]) for i in range(len(iris.data))]

In [ ]:
print(iris_samples[103].properties)

### Tests to split the training dataset
First we need to define all possible tests. 
In this dataset we have data collected across four properties for 150 samples. 
We will define tests for each of the four properties.
All properties have numerical data values so the natural question to ask about a given sample is whether the value for a property is greater than or less than some number $x$.
The choices of $x$ can be restricted to the values which appear in the dataset for this property.

If we had categorical data instead then it would not make sense to ask if a value is greater than or less than another value. 
For example, we may consider the color of each sample. 
In this case a slightly different type of test is required, but it is not hard to generalise the ideas from numerical to categorical data types.

We will create a test for each of the (up to) 150 values for each property.
The test can be expressed simply as the property which is being considered and the value of $x$ which will be used to split the data set into those samples which have a value less than or equal to $x$ for this property and those which have a value greater than $x$ for this property.

The function is defined so that if two identical tests are created only one copy is kept.

In [ ]:
NumericTest = namedtuple('NumericTest', ['property_index', 'value'])

def get_numeric_tests_from_samples(samples):
    """
        Define a test for each property of each sample.
        The test contains the property index and the value of this property for this sample.
        the idea is that we will later use these property values 
        to find a value that partitions the set of samples in a 'good' way 
        
        Parameters:
            samples: a list of objects of class Sample
            
        Returns:
            list of NumericTest tuples (property_index, value)
    """
    tests = set()
    for s in samples:
        for property_index, value in enumerate(s.properties):
            tests.add(NumericTest(property_index, value))
    return list(tests)

Now to decide which test is the best at splitting our dataset we need to look at how the test splits the training dataset.
The next function simply applies a test to a set of samples and returns the two resulting subsets.

In [ ]:
def split_samples_by_numeric_test(samples, test):
    """
        A test is a property index and value.
        We will test whether the relevant property is greater than the specified value 
        for each sample in samples.
        
        Parameters:
            samples: a list of objects of class Sample
            test: a NumericTest tuple (property_index, value).
            
        Returns:
            tuple of (leq_samples, gt_samples)
            where leq_samples: number of samples with relevant property not greater than the value given in test
            and gt_samples: number of samples with relevant property greater than the value given in test
    """
    leq_samples = []
    gt_samples = []
    for s in samples:
        if s.properties[test.property_index] > test.value:
            gt_samples.append(s)
        else:
            leq_samples.append(s)
    return leq_samples, gt_samples


The following cell is just an example of how these functions can be used. 
We get all of the tests appropriate for the iris dataset, and for the first of these check how it splits the data into two sets. 

In [ ]:
all_tests = get_numeric_tests_from_samples(iris_samples) 
example_test =  all_tests[0]

leq_samples, gt_samples = split_samples_by_numeric_test(iris_samples, example_test)

split_nums = (len(leq_samples), len(gt_samples))
split_nums

### Evaluating tests

The purpose of a decision tree is to split the data into subsets, each of which belong to the same classification. 
So to decide whether a test is 'good' we will look at the two subsets resulting from applying that test to our training data, in particular considering the classification that each sample belongs to.
Ideally we would like the subsets to each contain samples which all have the same classification. 
More generally we would like the subsets to show little variety with respect to the classification of the samples. 
The precise way to measure this is to calculate the entropy of the two sets of samples that result from applying a test. 

The next function calculates the entropy of a set of samples.
We will use this to calculate the average of the entropy of the two subsets resulting from a test. 

In [ ]:
def classification_frequency(samples):
    """
        Count the number of samples of each classification
        
        Parameters:
            samples: list of Sample objects
        
        Returns:
            dict {class_index: frequency} 
    """
    freq_dict = {}
    
    for s in samples:
        if s.class_index in freq_dict:
            freq_dict[s.class_index] += 1
        else:
            freq_dict[s.class_index] = 1
    
    return freq_dict

def classification_entropy(samples):
    """
        Given a list of Sample objects, group according to classification and get entropy of the distribution.
        
        Parameters:
            samples: list of Sample objects
        
        Returns:
            float, classification entropy of samples
    """
    freq_dict = classification_frequency(samples)
    freq_list = freq_dict.values()
    return calculate_entropy_from_outcome_distribution(freq_list)

In [ ]:
classification_entropy(iris_samples)

The next task is to consider the entropy from both subsets of samples following a test.
We average the two entropy values so that larger sample sets have more weight -- it is not helpful to have one very small set of samples with low entropy if the other much larger set has a high entropy. 

In [ ]:
def average_classification_entropy_after_test(samples, test):
    """
        Apply the given test to the samples
        Calculate the entropy of both resulting sets of samples
        and return the average of these.
        
        Parameters:
            samples: a list of Samples
            test: a NumericTest tuple
            
        Return:
            float, the average entropy after applying the test
    """
    leq, gt = split_samples_by_numeric_test(samples, test)
    num_leq = len(leq)
    num_gt = len(gt)
    expected = ((num_leq/(num_leq+num_gt)) * classification_entropy(leq)
                 +
                 (num_gt/(num_leq+num_gt)) * classification_entropy(gt))
    return expected        

Now that we have a method to calculate the average entropy following a test we can decide which test is most promising for our classification problem.

In the following function we make use of the expression `min(test_results, key = lambda x : x[1])`.
`test_results` is a list of tuples and there is no default ordering for tuples. Therefore the `key` parameter is provided to tell Python how to sort the tuples.

A `lambda` is an anonymous function.
```
lambda x : x[1]
```
is equivalent to  
``` 
def my_function(x):
        return x[1]
```
except that in the latter case the function has the name `my_function`.

Overall the expression `min(test_results, key = lambda x : x[1])` says to find the "minimum" valued tuple, where the value of each tuple is assumed to be the value it contains in position 1 (which is a numerical value, and therefore has a default ordering).

In [ ]:
def find_best_test_and_result(samples):
    """
        Get all tests based on the set of samples
        Determine the best test based on average entropy after applying it
        to these samples
        
        Parameters:
            samples: a list of Samples
            
        Returns:
            (NumericTest, average entropy)
    """
    tests = get_numeric_tests_from_samples(samples)
    ceat = average_classification_entropy_after_test
    test_results = [(test, ceat(samples, test)) for test in tests]
    
    # get the test with the best (ie lowest entropy) result
    # because we are sorting tuples we provide a rule for the sorting (key)
    # in this case we sort the tuples according to the value in position 1.
    best = min(test_results, key = lambda x : x[1])
    
    # return the test and result pair
    return best 

Note that there may be several tests with the same average entropy, our function does not explicitly state how this should be handled. Does it matter? What could be changed to make the choice explicit in the case of a tie in average entropy?

In [ ]:
find_best_test_and_result(iris_samples)

### Learning a decision tree
Now we have all the ingredients we need to create a decision tree from a set of training data. 
We will generate tests and find the average entropy of the two sample subsets resulting from each test. 
The test with the lowest associated average entropy is the best test at this point, so we split the samples accordingly and create another decision tree for each of the new sample sets. 

The process continues until we have a sample set containing all samples of the same type.
Alternatively, we stop generating further decision trees if there is no test which results in a lower entropy than the orignial sample set has. 

In [ ]:
def make_decision_tree(samples):
    """
        learn a decision tree from a set of training samples
        
        Parameters:
            samples: a list of Samples, the training data
            
        Return:
            a decision tree [test, left_tree, right_tree]
    """
    if samples == []:
        return "No Samples"
    
    sample_classes = classification_frequency(samples)
    
    # Check if all samples are in one class. Then we are done.
    if len(sample_classes) <= 1:
        return ["Classified", sample_classes]
    
    initial_entropy = classification_entropy(samples)
    test, entropy = find_best_test_and_result(samples)
    
    if entropy == initial_entropy:
        return ["Mixed", sample_classes]
    leq, gt = split_samples_by_numeric_test(samples,test)
    
    # Make decision trees for samples remaining after each of
    # the test outcomes (<= and >)
    left_tree = make_decision_tree(leq)
    right_tree = make_decision_tree(gt)
    return [test, left_tree, right_tree]

In [ ]:
def display_decision_tree(dt, class_names, property_names, prefix='', indent=0):
    """
        print a decision tree in a readable way.
        
        Parameters:
            dt: the tree to be printed
            class_names: names of target classes for the data used to train dt
            property_names: names of properties for the data used to train dt
            prefix: a strong to be printed before the tree
            indent: amount of indentation before printing the tree
        
        Returns:
            None
    """
    print(" "*6*indent, prefix, end='')
    if len(dt) < 3:
        classification = [class_names[k]+' ({})'.format(v) for k,v in dt[1].items()]
        print(dt[0], ','.join(classification))
    else:
        test = dt[0]
        print( "test:", property_names[test[0]], test[1])
        display_decision_tree(dt[1], class_names, property_names, 'leq - ', indent+1)
        display_decision_tree(dt[2], class_names, property_names, 'gt - ', indent+1)       

We make a decision tree for the iris data.
We will first remove one sample of each class to test the decision tree.

In [ ]:
test_set = []
training_set = []
class_indices = []

for i, s in enumerate(iris_samples):
    if s.class_index not in class_indices:
        test_set.append(s)
        class_indices.append(s.class_index)
    else:
        training_set.append(s)

In [ ]:
dt = make_decision_tree(training_set)
display_decision_tree(dt, iris.target_names, iris.feature_names)

### Using the decision tree
Now we want to use the decision tree to classify our three testing samples. 

In [ ]:
def classify_with_decision_tree(tree, samples):
    """
        Classify the given sample using the decision tree.
        
        Parameters:
            tree: decision tree (test, left_tree, right_tree)
            samples: a list of Sample objects
            
        Return:
            a list of predicted classes
    """
    predictions = []
    for s in samples:
        curr_tree = tree
        
        # apply decision trees recursively
        while len(curr_tree) == 3:
            test = curr_tree[0]
            if s.properties[test.property_index] <= test.value:
                curr_tree = curr_tree[1]
            else:
                curr_tree = curr_tree[2]
        
        # base case - return most common training class for this node
        node_classes = curr_tree[1]
        predictions.append(max(node_classes, key=node_classes.get))
    
    return predictions

Hopefully the decision tree will classify the three samples into classes 0 (setosa), 1 (versicolor) and 2 (virginica) respectively.
If so, this suggests that the decision tree has found a useful way of classifying types of flower based on the four measurements given, and that this classification works beyond the data that was used to create the tree.

In [ ]:
classify_with_decision_tree(dt, test_set)

Now that you have seen how to make a decision tree classifier you may be interested to have a look at the scikit-learn documentation about [decision trees](https://scikit-learn.org/stable/modules/tree.html#tree) to find out more.

### Overfitting
One thing to consider from this example is the problem of overfitting, which is when a learning technique fails to generalise from the data effectively.
The outcome is a classifier that works well for the training data but is not reliable otherwise.
Overfitting also creates trees which are too large.

Notice that all of the leaves in our decision trees contain just a single type of flower. 
That's good, and is the purpose of the classifier.
However, some of them achieve this by containing just one sample! 
The tests which led to this node only applied to a single sample in the training set, so we have some reason to suspect that they are not good tests in general for recognising this type of flower. 

Many leaves with a very small number of samples is a sign of possible overfitting.
One way to reduce overfitting is to insist on a minimum number of samples associated with each leaf node. 
You could rewrite the functions above to allow a minimum to be specified and retrain the decision tree. 
Scikit-learn's decision tree classifier has an optional parameter for a minimum leaf size.

Another way of reducing overfitting is to set a maximum depth for the tree. 
In this case, because the data set was small, the most tests that can be required to classify a sample is 5. 
With larger datasets decision trees can become very large.
We can control this by enforcing that the number of tests between the start of the decision tree and any leaf node does not exceed some given value.